# Importing Libraries

In [ ]:
from ipywidgets import interact, Dropdown
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import sklearn as sk
from sklearn import model_selection as skms
from sklearn import datasets
from sklearn import preprocessing as skpp

import os
from tensorflow.python.framework import ops
ops.reset_default_graph()
import traceback
from tensorflow import keras

from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense

from sklearn import datasets
from scipy.spatial import cKDTree
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

# Green House Gases Emmision Data

## Reading Data

In [ ]:
# Climate Analysis Indicators Tool (CAIT) 
cait=r"data\CW_CAIT_GHG_Emissions.xlsx'
cait = pd.ExcelFile(cait)
# Potsdam Institute for Climate Impact Research (PIK) 
pik=r"data\CW_PIK_GHG_Emissions.xlsx'
pik = pd.ExcelFile(pik)

cait_data = {sheet_name: cait.parse(sheet_name) 
          for sheet_name in cait.sheet_names}
cait_data = cait_data['CAIT_2019_update']
cait_data.columns=[str(j) for j in cait_data.columns]
cait_data.columns = cait_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
pik_data = {sheet_name: pik.parse(sheet_name) 
          for sheet_name in pik.sheet_names}
pik_data=pik_data['CW_HistoricalEmissions_PIK_2019']
pik_data.columns=[str(j) for j in pik_data.columns]
pik_data.columns = pik_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

## Univariate MLP Timeseries Forecasting

In [ ]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def keras_predict(listt, x_input):
    raw_seq=listt
    n_steps = 3
    X, y = split_sequence(raw_seq, n_steps)
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim=n_steps))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=2000, verbose=0)
    x_input = array(x_input)
    x_input = x_input.reshape((1, n_steps))
    yhat = model.predict(x_input, verbose=0)
    return yhat

## Data Selection

In [ ]:
dataaw = Dropdown(options = ['CAIT', 'PIK'])

@interact(file = dataaw)
def gas_data(file):  
    if (file == 'CAIT'):
        dataa = cait_data.copy()
    else:
        dataa = pik_data.copy()    
    country_drop = Dropdown(options = list(set(list(dataa['country']))))
    sector_drop = Dropdown(options = list(set(list(dataa['sector']))))
    gas_drop = Dropdown(options = list(set(list(dataa['gas']))))
    @interact(country_name=country_drop, sec_name = sector_drop, gas_name=gas_drop)
    def print_city(country_name, sec_name, gas_name):
        print('country_name:',country_name , sep=' ')
        print( 'sec_name:',sec_name , sep=' ')
        print( 'gas_name:',gas_name,  sep=' ')
        try:
            ts_data=dataa[(dataa['country']==country_name) & (dataa['sector']==sec_name) & (dataa['gas']==gas_name)].transpose()[4:]
            ts_data['year']=ts_data.index
            ts_data.reset_index(drop=True, inplace=True)
            ts_data.columns=['value', 'year']
            ts_data.dropna(subset = ["value"], inplace=True)
            print('TimeSeries  Data:=')
            print(ts_data.transpose())
            values_list = list(ts_data['value'])
            print('Next Value could be', 
                  keras_predict(values_list,values_list[-3:] ))
        except:
            print('Data Not Available')
            return False
#         ts_data=dataa[dataa['']]

interactive(children=(Dropdown(description='file', options=('CAIT', 'PIK'), value='CAIT'), Output()), _dom_cla…

## Simple Single Layer Neural Network Implementation

In [ ]:
def normalize_cols(m):
    col_max = m.max(axis=0)
    col_min = m.min(axis=0)
    return (m-col_min) / (col_max - col_min)
def single_neu_net(np_array):
    x_vals = np.array([x[0:3] for x in np_array])
    y_vals = np.array([x[3] for x in np_array])
    sess = tf.Session()
    seed = 2
    tf.set_random_seed(seed)
    np.random.seed(seed)  
    train_indices = np.random.choice(len(x_vals), round(len(x_vals)*0.8), replace=False)
    test_indices = np.array(list(set(range(len(x_vals))) - set(train_indices)))
    x_vals_train = x_vals[train_indices]
    x_vals_test = x_vals[test_indices]
    y_vals_train = y_vals[train_indices]
    y_vals_test = y_vals[test_indices]
    x_vals_train = np.nan_to_num(normalize_cols(x_vals_train))
    x_vals_test = np.nan_to_num(normalize_cols(x_vals_test))
    batch_size = 50
    x_data = tf.placeholder(shape=[None, 3], dtype=tf.float32)
    y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
    y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
    hidden_layer_nodes = 10
    A1 = tf.Variable(tf.random_normal(shape=[3,hidden_layer_nodes])) # inputs -> hidden nodes
    b1 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes]))   # one biases for each hidden node
    A2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes,1])) # hidden inputs -> 1 output
    b2 = tf.Variable(tf.random_normal(shape=[1]))   # 1 bias for the output
    hidden_output = tf.nn.relu(tf.add(tf.matmul(x_data, A1), b1))
    final_output = tf.nn.relu(tf.add(tf.matmul(hidden_output, A2), b2))
    loss = tf.reduce_mean(tf.square(y_target - final_output))
    my_opt = tf.train.GradientDescentOptimizer(0.005)
    train_step = my_opt.minimize(loss)
    init = tf.global_variables_initializer()
    sess.run(init)
    loss_vec = []
    test_loss = []
    for i in range(500):
        rand_index = np.random.choice(len(x_vals_train), size=batch_size)
        rand_x = x_vals_train[rand_index]
        rand_y = np.transpose([y_vals_train[rand_index]])
        sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})

        temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
        loss_vec.append(np.sqrt(temp_loss))

        test_temp_loss = sess.run(loss, feed_dict={x_data: x_vals_test, y_target: np.transpose([y_vals_test])})
        test_loss.append(np.sqrt(test_temp_loss))
        if (i+1)%50==0:
            print('Generation: ' + str(i+1) + '. Loss = ' + str(temp_loss))
    get_ipython().run_line_magic('matplotlib', 'inline')
    plt.plot(loss_vec, 'k-', label='Train Loss')
    plt.plot(test_loss, 'r--', label='Test Loss')
    plt.title('Loss (MSE) per Generation')
    plt.legend(loc='upper right')
    plt.xlabel('Generation')
    plt.ylabel('Loss')
    plt.show()


## Data Selection

In [ ]:
dataaw = Dropdown(options = ['CAIT', 'PIK'])

@interact(file = dataaw)
def gas_data(file):  
    if (file == 'CAIT'):
        dataa = cait_data.copy()
    else:
        dataa = pik_data.copy()
#     print(dataa)
    sector_drop = Dropdown(options = list(set(list(dataa['sector']))))
    gas_drop = Dropdown(options = list(set(list(dataa['gas']))))
    year_drop = Dropdown(options = list(dataa.columns[4:]))
    @interact(year=year_drop, sec_name = sector_drop, gas_name=gas_drop)
    def print_city(year, sec_name, gas_name):
        print('year:',year , sep=' ')
        print( 'sec_name:',sec_name , sep=' ')
        print( 'gas_name:',gas_name,  sep=' ')

        try:
            cols=['country', 'sector', 'gas' ]
            for j in cols:
                cons=list(set(list(dataa[j])))
                con_dict={cons[i]:i for i in range(len(cons))}
                dataa[j]=[con_dict[i] for i in dataa[j]]

            new_data = dataa[[ 'gas', 'sector',year, 'country',]].copy()
            print(new_data)
            single_neu_net(new_data.to_numpy())
        except:
            print('Data Not Available')
            return False

interactive(children=(Dropdown(description='file', options=('CAIT', 'PIK'), value='CAIT'), Output()), _dom_cla…

# CW_WB_Climate_Vulnerability_Readiness Analysis

## Reading Data

In [ ]:
xl_file = pd.ExcelFile(r"data\CW_WB_Climate_Vulnerability_Readiness_31102017.xlsx")

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}
dataa=dfs['Vulnerability_Readiness']
col_list=['country',
          'Population living below the national income poverty line',
          'Climate Risk Score',
          'Vulnerability score',
          'Readiness score',
          'Link to climate risk and adaptation profile',
          'Droughts',
          'Storms',
          'Epidemics',
          'Landslides',
          'Floods',
          'Wildfires'
]

dataa.columns=col_list
dataa.columns=[str(j) for j in dataa.columns]
dataa.columns = dataa.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
dataa.dropna(subset = ["country"], inplace=True)
dataa.dropna(subset = ["readiness_score"], inplace=True)
dataa=dataa.drop(["link_to_climate_risk_and_adaptation_profile"], axis=1)
dataa=dataa[3:].copy().reset_index().drop(['index'], axis=1)
for i in dataa.columns[5:]:
    dataa[i] = dataa[i].fillna(-1)
    dataa[i] = dataa[i].replace(['YES'],1)
    dataa[i] = dataa[i].replace(['NO'],-1)
con_dict = {dataa['country'][i]:i for i in range(len(dataa['country']))}
print("Data Preview:-")
print(dataa.head(5))

Data Preview:-
  country population_living_below_the_national_income_poverty_line  \
0     AGO                                                NaN         
1     ALB                                               14.3         
2     ARE                                                NaN         
3     ARG                                                NaN         
4     ARM                                                 30         

  climate_risk_score vulnerability_score readiness_score  droughts  storms  \
0             106.67            0.566557        0.289926        -1      -1   
1              125.5            0.386407        0.525043        -1      -1   
2             154.67            0.364053        0.603236        -1      -1   
3              84.67             0.37448        0.440038        -1      -1   
4              139.5            0.384046        0.537698        -1      -1   

   epidemics  landslides  floods  wildfires  
0         -1          -1      -1         -1  
1  

## Gaussion SVM Implementation

In [ ]:
def gaus_svm(tonumpy_data, data_ori, features):
    dataa = tonumpy_data
    x_vals = np.array([[x[0], x[1]] for x in dataa])
    y_vals = np.array(list(data_ori[features[2]]))
    class1_x = [x[0] for i,x in enumerate(x_vals) if y_vals[i]==1]
    class1_y = [x[1] for i,x in enumerate(x_vals) if y_vals[i]==1]
    class2_x = [x[0] for i,x in enumerate(x_vals) if y_vals[i]==-1]
    class2_y = [x[1] for i,x in enumerate(x_vals) if y_vals[i]==-1]
    sess = tf.Session()
    batch_size = len(x_vals)
    x_data = tf.placeholder(shape=[None, 2], dtype=tf.float32)
    y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
    prediction_grid = tf.placeholder(shape=[None, 2], dtype=tf.float32)
    b = tf.Variable(tf.random_normal(shape=[1,batch_size]))
    gamma = tf.constant(-50.0)
    sq_vec = tf.multiply(2., tf.matmul(x_data, tf.transpose(x_data)))
    my_kernel = tf.exp(tf.multiply(gamma, tf.abs(sq_vec)))
    first_term = tf.reduce_sum(b)
    b_vec_cross = tf.matmul(tf.transpose(b), b)
    y_target_cross = tf.matmul(y_target, tf.transpose(y_target))
    second_term = tf.reduce_sum(tf.multiply(my_kernel, tf.multiply(b_vec_cross, y_target_cross)))
    loss = tf.negative(tf.subtract(first_term, second_term))
    rA = tf.reshape(tf.reduce_sum(tf.square(x_data), 1),[-1,1])
    rB = tf.reshape(tf.reduce_sum(tf.square(prediction_grid), 1),[-1,1])
    pred_sq_dist = tf.add(tf.subtract(rA, tf.multiply(2., tf.matmul(x_data, tf.transpose(prediction_grid)))), tf.transpose(rB))
    pred_kernel = tf.exp(tf.multiply(gamma, tf.abs(pred_sq_dist)))
    prediction_output = tf.matmul(tf.multiply(tf.transpose(y_target),b), pred_kernel)
    prediction = tf.sign(prediction_output-tf.reduce_mean(prediction_output))
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.squeeze(prediction), tf.squeeze(y_target)), tf.float32))
    my_opt = tf.train.GradientDescentOptimizer(0.01)
    train_step = my_opt.minimize(loss)
    init = tf.global_variables_initializer()
    sess.run(init)
    loss_vec = []
    batch_accuracy = []
    for i in range(300):
        rand_index = np.random.choice(len(x_vals), size=batch_size)
        rand_x = x_vals[rand_index]
        rand_y = np.transpose([y_vals[rand_index]])
        sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})

        temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
        loss_vec.append(temp_loss)

        acc_temp = sess.run(accuracy, feed_dict={x_data: rand_x,
                                                 y_target: rand_y,
                                                 prediction_grid:rand_x})
        batch_accuracy.append(acc_temp)

        if (i+1)%75==0:
            print('Step #' + str(i+1))
            print('Loss = ' + str(temp_loss))
    x_min, x_max = x_vals[:, 0].min() - 1, x_vals[:, 0].max() + 1
    y_min, y_max = x_vals[:, 1].min() - 1, x_vals[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))
    grid_points = np.c_[xx.ravel(), yy.ravel()]
    [grid_predictions] = sess.run(prediction, feed_dict={x_data: x_vals,
                                                       y_target: np.transpose([y_vals]),
                                                       prediction_grid: grid_points})
    grid_predictions = grid_predictions.reshape(xx.shape)
    get_ipython().run_line_magic('matplotlib', 'inline')
    plt.contourf(xx, yy, grid_predictions, cmap=plt.cm.Paired, alpha=0.8)
    plt.plot(class1_x, class1_y, 'ro', label='Immune')
    plt.plot(class2_x, class2_y, 'kx', label='Not Immune')
    plt.title('Gaussian SVM Results on Climate_Vulnerability_Readiness Data for '+features[2])
    plt.xlabel(features[0])
    plt.ylabel(features[1])
    plt.legend(loc='lower right')
    plt.show()
    plt.plot(batch_accuracy, 'k-', label='Accuracy')
    plt.title('Batch Accuracy')
    plt.xlabel('Generation')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.show()
    plt.plot(loss_vec, 'k-')
    plt.title('Loss per Generation')
    plt.xlabel('Generation')
    plt.ylabel('Loss')
    plt.show()


## Data Selection

In [ ]:
Feature1 = Dropdown(options = ['climate_risk_score','vulnerability_score','readiness_score'])
Feature2 = Dropdown(options = ['climate_risk_score','vulnerability_score','readiness_score'])
Calamity = Dropdown(options = ['droughts','storms','epidemics','landslides','floods','wildfires'])
@interact(f1=Feature1, f2=Feature2, cal=Calamity)
def clim_vul_data(f1, f2, cal):
    if (f1==f2):
        print('same Feature Selected, Please Select Different one')
        return False
    new_data = dataa.filter([f1,f2,cal], axis=1)
    for i in new_data.columns:
        new_data[i] = new_data[i].fillna(0) 
    gaus_svm(new_data.to_numpy(), new_data, [f1,f2,cal])

interactive(children=(Dropdown(description='f1', options=('climate_risk_score', 'vulnerability_score', 'readin…

# PathWays Data

## Reading Data

In [ ]:
gcam = pd.ExcelFile(r"data\GCAM.xlsx")
gcam = {sheet_name: gcam.parse(sheet_name) 
          for sheet_name in gcam.sheet_names}
gcam = gcam['GCAM_Timeseries data']
ieo = pd.ExcelFile(r"data\IEO.xlsx")
ieo = {sheet_name: ieo.parse(sheet_name) 
          for sheet_name in ieo.sheet_names}
ieo = ieo['IEO_Timeseries data']

df_list = [gcam, ieo]
for dfs in df_list:
    dfs.columns=[str(j) for j in dfs.columns]
    dfs.columns = dfs.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


## Multivariate MLP TimeSeries Forecasting

In [ ]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def multivar_timeseries(listt):
    list1=listt[0]
    list2=listt[1]
    in_seq1 = array(list1)
    in_seq2 = array(list2)
    out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    out_seq = out_seq.reshape((len(out_seq), 1))
    dataset = hstack((in_seq1, in_seq2, out_seq))
    n_steps = 3
    X, y = split_sequences(dataset, n_steps)
    n_input = X.shape[1] * X.shape[2]
    X = X.reshape((X.shape[0], n_input))
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim=n_input))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=2000, verbose=0)
    # x_input = array([[80, 85], [90, 95], [100, 105]])
    x_input = array([list1[-3:],list2[-3:]]).transpose()
    x_input = x_input.reshape((1, n_input))
    yhat = model.predict(x_input, verbose=0)
    return yhat

## Data Selection

In [ ]:
typee = Dropdown(options = ['GCAM', 'IEO'])
@interact(typee=typee)
def clim_vul_data(typee):
    dataa=globals()[typee.lower()]
    dataa=dataa.fillna(0)
    scenario = Dropdown(options = dataa.scenario.unique())
    indi_name = Dropdown(options = dataa.esp_indicator_name.unique())
    region1 = Dropdown(options = dataa.region.unique())
    region2 = Dropdown(options = dataa.region.unique())
    @interact(region1=region1, region2=region2, scenario=scenario, indi_name=indi_name)
    def in_data(scenario, indi_name, region1, region2):
        print('scenario:',scenario,'\nesp_indicator_name:', indi_name,'\nregion1:', region1,'\nregion2:', region2)
        print('Unit:', dataa[dataa['esp_indicator_name']==indi_name].unit_of_entry.unique())
        r1_ts=dataa[(dataa['scenario']==scenario) & (dataa['esp_indicator_name']==indi_name) & (dataa['region'] == region1) ]
        r2_ts=dataa[(dataa['scenario']==scenario) & (dataa['esp_indicator_name']==indi_name) & (dataa['region'] == region2) ]
        reg_df=[r1_ts, r2_ts]
        for i in range(len(reg_df)):
            reg_df[i]=reg_df[i].transpose()[5:]
            reg_df[i]['year']=reg_df[i].index
            reg_df[i].reset_index()
            reg_df[i].columns=['value', 'year']
            print('Region', str(i+1), 'Series:=')
            print(reg_df[i].transpose())
        try:
            print('Next Value for both could be', str(multivar_timeseries([list(i['value']) for i in reg_df])/2))
        except:
            print('Series Inconsistent!')



interactive(children=(Dropdown(description='typee', options=('GCAM', 'IEO'), value='GCAM'), Output()), _dom_cl…

# socioeconomic_indicators_GDP Data

## Reading Data

In [ ]:
soc_eco = pd.ExcelFile(r"data\socioeconomic_indicators_GDP.xlsx")
soc_eco = {sheet_name: soc_eco.parse(sheet_name) 
          for sheet_name in soc_eco.sheet_names}
soc_eco = soc_eco['socioeconomic_indicators_GDP']
col_list=[]
for i in soc_eco.iloc[[1]].columns:
    col_list.append(list(soc_eco.iloc[[1]][i])[0])
col_list=[str(j) for j in col_list]
col_list = [i.strip().lower().replace(' ', '_').replace('(', '').replace(')', '') for i in col_list]
units=[]
for i in soc_eco.iloc[[2]].columns:
    units.append(list(soc_eco.iloc[[2]][i])[0])
soc_eco.columns=col_list
unit_dict=dict()
for i in range(len(units)):
    unit_dict[col_list[i]]=units[i] 
soc_eco=soc_eco[3:]
soc_eco=soc_eco.reset_index()
soc_eco=soc_eco.drop(['index'], axis = 1) 
soc_eco=soc_eco.fillna(0)

## Simple KERAS Model Experimentation

In [ ]:
year = Dropdown(options = ['2011', '2012','2013','2014','2015','2016'])
asd=[]
@interact(year=year)
def soc_eco_fun(year):
    res = [i for i in soc_eco.columns if year in i] 
    print(res)
    columnss=list(soc_eco.columns[2:])
    pred_ft = Dropdown(options = list([item for item in columnss if item not in res]))
    @interact(pred_ft=pred_ft)
    def pred_var(pred_ft):
        print('Prediction Column:', pred_ft)
        new_list=res+[pred_ft]
        new_df=soc_eco[new_list].copy()
        new_df[pred_ft] = new_df[pred_ft].astype('category')
        new_df[pred_ft] = new_df[[pred_ft]].apply(lambda x: x.cat.codes)
        features = new_df.drop(pred_ft, axis=1).to_numpy()
        labels = new_df[pred_ft].to_numpy()
        train_features = features[:-1, :]
        test_features = features[-1:]
        train_labels = labels[:-1]
        test_labels = labels[-1:]
        model = keras.Sequential([
            keras.layers.Dense(256, activation='relu'),
            keras.layers.Dense(5, activation='softmax')
        ])
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        model.fit(train_features, train_labels, epochs=15, validation_split=0.3)
        print('Model Prediction : ',model.predict(test_features))


interactive(children=(Dropdown(description='year', options=('2011', '2012', '2013', '2014', '2015', '2016'), v…

# CW_Agriculture_Profile Analysis

## Reading Data

In [1]:
xl_file = pd.ExcelFile(r"data\CW_Agriculture_Profile.xlsx")

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}
for i in dfs.keys():
    dfs[i].columns=[str(j) for j in dfs[i].columns]
    dfs[i].columns = dfs[i].columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

sheets=list(dfs.keys())

NameError: ignored

## Simple Tensorflow Regression Implementation

In [2]:
#linear regression
def tens_lin_reg(year, value):
    x_vals = np.array([float(i) for i in year])
    y_vals = np.array(value)
    sess = tf.Session()
    batch_size = 10
    tf.compat.v1.disable_eager_execution()
    x_data = tf.compat.v1.placeholder(shape=[None, 1], dtype=tf.float32)
    y_target = tf.compat.v1.placeholder(shape=[None, 1], dtype=tf.float32)
    A = tf.Variable(tf.random.normal(shape=[1,1]))
    b = tf.Variable(tf.random.normal(shape=[1,1]))
    model_output = tf.add(tf.matmul(x_data, A), b)
    loss = tf.reduce_mean(tf.square(y_target - model_output))
    tf.disable_v2_behavior()
    my_opt=tf.train.GradientDescentOptimizer(learning_rate=0.00000009)
    train_step = my_opt.minimize(loss)
    init = tf.global_variables_initializer()
    sess.run(init)
    loss_vec = []
    for i in range(500):
        rand_index = np.random.choice(len(x_vals), size=batch_size)
        rand_x = np.transpose([x_vals[rand_index]])
        rand_y = np.transpose([y_vals[rand_index]])
        _, temp_loss=sess.run([train_step,loss], feed_dict={x_data: rand_x, y_target: rand_y})
        loss_vec.append(temp_loss)
        if (i+1)%100==0:
            print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)) + ' b = ' + str(sess.run(b)))
            print('Loss = ' + str(temp_loss))
    [slope] = sess.run(A)
    [y_intercept] = sess.run(b)
    best_fit = []
    for i in x_vals:
      best_fit.append(slope*i+y_intercept)
    fig, axs = plt.subplots(2)
    axs[0].plot(x_vals, y_vals, 'o', label='Data Points')
    axs[0].plot(x_vals, best_fit, 'r-', label='Best fit line', linewidth=3)
    axs[0].legend(loc='upper left')
    axs[1].plot(loss_vec, 'k-', label='Loss Per Generation')
    # axs[1].show()
    fig.suptitle('Data Regression Plot')
    fig.set_figheight(10)
    fig.set_figwidth(20)


## Data Selection

In [3]:
dataaw = Dropdown(options = list(dfs.keys())[1:])

@interact(file = dataaw)
def asdasd(file):  
    emmisions=dict()
    c_var='area'
    try:
        emmisions_keys=set(list(dfs[file]['area']))
    except:
        c_var='country'
        emmisions_keys=set(list(dfs[file][c_var]))
    for i in emmisions_keys:
        emmisions[i]=list(set(list(dfs[file][dfs[file][c_var]==i]['short_names'])))

#     print(emmisions)
    countryW = Dropdown(options = emmisions.keys())
    typee = Dropdown()

    def update_typee_options(*args): 
        typee.options = emmisions[countryW.value]
    typee.observe(update_typee_options) 

    @interact(country = countryW, emm_type = typee)
    def print_city(country, emm_type):
        print('Emmision Type Summary:')
        print(dfs[sheets[0]][dfs[sheets[0]][list(dfs[sheets[0]].columns)[0]]==emm_type].transpose())
        try:
            print('\nFileName: '+file)
            print('Country:' +country+'\nType: '+emm_type+'\n')
            ts=dfs[file][(dfs[file][c_var]==country)&(dfs[file]['short_names']==emm_type)].transpose()
            ts=ts[list(ts.index).index("short_names")+1:]
            ts['year']=ts.index
            ts.reset_index(drop=True, inplace=True)
            ts.columns=['value', 'year']
            ts.dropna(subset = ["value"], inplace=True)
            ts['year']=[int(i) for i in ts['year']]
            tens_lin_reg(ts['year'], ts['value'])
        except:
            print('TimeSeries Data Inconsistent!')
#             return False

NameError: ignored